In [7]:
import json
import os


input_dir = '../gym/json/'

### JSON example
# {
#     "PAPER'S NUMBER OF TABLES": 5,   to be removed
#     "S1.T1": {                       the table_id
#         "caption": "Table 1. Existing",
#         "table": "<table id=\"S1.T1.1.1\" +</tr>\n</table>\n\n",
#         "footnotes": [
#             "1"
#         ],
#         "references": [
#             "A",
#         ],
#     },
# }

def read_json(file):
    with open(file, 'r') as f:
        data = json.load(f)
    return data


In [8]:
# Creating a table with table_id, caption, footnotes, references

import pandas as pd

df = pd.DataFrame(columns=['index, table_id', 'caption', 'references'])

for file in os.listdir(input_dir):
    data = read_json(input_dir + file)
    for table_id in data:
        if table_id == "PAPER'S NUMBER OF TABLES":
            continue
        table = data[table_id]
        caption = table['caption']
        references = table['references']
        df.loc[len(df)] = [table_id, caption, references]


In [9]:
df.to_csv('data.csv', index=True)